In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.src.metrics import accuracy


2024-10-21 15:28:29.784287: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-21 15:28:29.985562: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-21 15:28:29.985584: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-21 15:28:29.986705: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-21 15:28:30.100400: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-21 15:28:30.101726: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
from folktables import ACSDataSource, ACSPublicCoverage
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')

# Load Data for All US States

In [3]:
# List of all 50 US state abbreviations
states = [
    "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA",
    "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD",
    "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ",
    "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
    "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"
]

# Dictionary to store data, features, and labels for each state
state_data = {}
state_features = {}
state_labels = {}
state_groups = {}

# Assuming `data_source` and `ACSIncome` are already defined
for state in states:
    # Load data for each state
    state_data[state] = data_source.get_data(states=[state], download=True)
    # Convert to features and labels
    state_features[state], state_labels[state], state_groups[state]= ACSPublicCoverage.df_to_numpy(state_data[state])
    state_features[state] = scaler.fit_transform(state_features[state])
    state_groups[state] = np.where(state_groups[state] == 1, 0, 1)
    print(state, state_features[state].shape, state_labels[state].shape,state_groups[state].shape)

# Example usage: Access data, features, and labels for California (CA)
ca_features = state_features["CA"]
ca_labels = state_labels["CA"]


AL (18525, 19) (18525,) (18525,)
AK (2377, 19) (2377,) (2377,)
AZ (24988, 19) (24988,) (24988,)
AR (11838, 19) (11838,) (11838,)
CA (138554, 19) (138554,) (138554,)
CO (17630, 19) (17630,) (17630,)
CT (11204, 19) (11204,) (11204,)
DE (2871, 19) (2871,) (2871,)
FL (71297, 19) (71297,) (71297,)
GA (37701, 19) (37701,) (37701,)
HI (4660, 19) (4660,) (4660,)
ID (5937, 19) (5937,) (5937,)
IL (42827, 19) (42827,) (42827,)
IN (24330, 19) (24330,) (24330,)
IA (10241, 19) (10241,) (10241,)
KS (10242, 19) (10242,) (10242,)
KY (17166, 19) (17166,) (17166,)
LA (16879, 19) (16879,) (16879,)
ME (4513, 19) (4513,) (4513,)
MD (18025, 19) (18025,) (18025,)
MA (21992, 19) (21992,) (21992,)
MI (35624, 19) (35624,) (35624,)
MN (16357, 19) (16357,) (16357,)
MS (11831, 19) (11831,) (11831,)
MO (22327, 19) (22327,) (22327,)
MT (3519, 19) (3519,) (3519,)
NE (6332, 19) (6332,) (6332,)
NV (10201, 19) (10201,) (10201,)
NH (4251, 19) (4251,) (4251,)
NJ (27609, 19) (27609,) (27609,)
NM (7693, 19) (7693,) (7693,)
N

# Split Data into Train, Validation, and Test 

In [4]:
from sklearn.model_selection import train_test_split
# Define dictionaries to store train, validation, and test data for each state
train_features = {}
train_labels = {}
train_groups = {}
val_features = {}
val_labels = {}
vali_groups = {}
test_features = {}
test_labels = {}
test_groups = {}
all_test_features = []
all_test_labels = []
all_test_groups = []
all_vaild_features = []
all_valid_labels = []
all_valid_groups = []
from sklearn.model_selection import train_test_split


# Split ratio configuration
train_ratio = 0.5
val_ratio = 0.3
test_ratio = 0.2

for state in states:
    # Get the features and labels for the state
    features = state_features[state]
    labels = state_labels[state]
    groups = state_groups[state]

    # First split into train and temp (validation + test) sets
    X_train, X_temp, y_train, y_temp, s_train, s_temp = train_test_split(
        features, labels, groups, test_size=(1 - train_ratio), random_state=42)

    # Then split the temp set into validation and test sets
    X_val, X_test, y_val, y_test, s_val, s_test = train_test_split(
        X_temp, y_temp, s_temp, test_size=(test_ratio / (val_ratio + test_ratio)), random_state=42)

    # Store the splits into respective dictionaries
    train_features[state] = X_train
    train_labels[state] = y_train
    train_groups[state] = s_train
    val_features[state] = X_val
    val_labels[state] = y_val
    vali_groups[state] = s_val
    test_features[state] = X_test
    test_labels[state] = y_test
    test_groups[state] = s_test
    all_vaild_features.append(X_val)
    all_valid_labels.append(y_val)
    all_valid_groups.append(s_val)
    all_test_features.append(X_test)
    all_test_labels.append(y_test)
    all_test_groups.append(s_test)

    # Print the shapes for verification
    print(f"{state}: Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
all_test_features = np.concatenate(all_test_features)
all_test_labels = np.concatenate(all_test_labels)
all_test_groups = np.concatenate(all_test_groups)
all_vaild_features=np.concatenate(all_vaild_features)
all_vaild_labels=np.concatenate(all_valid_labels)
all_valid_groups=np.concatenate(all_valid_groups)




AL: Train: (9262, 19), Val: (5557, 19), Test: (3706, 19)
AK: Train: (1188, 19), Val: (713, 19), Test: (476, 19)
AZ: Train: (12494, 19), Val: (7496, 19), Test: (4998, 19)
AR: Train: (5919, 19), Val: (3551, 19), Test: (2368, 19)
CA: Train: (69277, 19), Val: (41566, 19), Test: (27711, 19)
CO: Train: (8815, 19), Val: (5289, 19), Test: (3526, 19)
CT: Train: (5602, 19), Val: (3361, 19), Test: (2241, 19)
DE: Train: (1435, 19), Val: (861, 19), Test: (575, 19)
FL: Train: (35648, 19), Val: (21389, 19), Test: (14260, 19)
GA: Train: (18850, 19), Val: (11310, 19), Test: (7541, 19)
HI: Train: (2330, 19), Val: (1398, 19), Test: (932, 19)
ID: Train: (2968, 19), Val: (1781, 19), Test: (1188, 19)
IL: Train: (21413, 19), Val: (12848, 19), Test: (8566, 19)
IN: Train: (12165, 19), Val: (7299, 19), Test: (4866, 19)
IA: Train: (5120, 19), Val: (3072, 19), Test: (2049, 19)
KS: Train: (5121, 19), Val: (3072, 19), Test: (2049, 19)
KY: Train: (8583, 19), Val: (5149, 19), Test: (3434, 19)
LA: Train: (8439, 19), V

# Train a FedAvg model

In [5]:
import tensorflow as tf
import numpy as np
import time

# List of all 50 US state abbreviations

# Define a simple neural network model
def create_model(input_shape):
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=input_shape),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Training parameters
global_epochs = 10
local_epochs = 1
batch_size = 256

# Initialize global model
input_shape = state_features[states[0]].shape[1:]  # Assuming all states have the same feature shape
global_model = create_model(input_shape)

# Federated Averaging
for epoch in range(global_epochs):
    start_time=time.time()
    print(f"Global Epoch {epoch + 1}/{global_epochs}")
    
    # Store the weights from each local model
    local_weights = []

    # Train model on each state's data
    for state in states:
        # print(f"Training on state: {state}")

        # Create a local model for each state
        local_model = create_model(input_shape)
        local_model.set_weights(global_model.get_weights())  # Initialize with global model weights

        # Train local model
        local_model.fit(
            state_features[state], state_labels[state],
            epochs=local_epochs, batch_size=batch_size
        )

        # Append the local weights
        local_weights.append(local_model.get_weights())

    # Average the weights to get new global weights
    new_global_weights = [np.mean([local_weights[j][i] for j in range(len(local_weights))], axis=0)
                          for i in range(len(local_weights[0]))]
    end_time=time.time()
    # print('time:',end_time-start_time)
    # Set the new global weights
    global_model.set_weights(new_global_weights)
    loss,accuracy=global_model.evaluate(all_vaild_features,all_vaild_labels)
    print('validation accuray:',accuracy)
# Evaluate the global model
print("\nEvaluating Global Model")
for state in states:
    loss, accuracy = global_model.evaluate(val_features[state], val_labels[state], verbose=0)
    print(f"{state} - Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")




2024-10-21 15:29:09.057746: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-21 15:29:09.210549: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Global Epoch 1/10
10532/10532 [==============================] - 5s 505us/step - loss: 0.5316 - accuracy: 0.7591
validation accuray: 0.7591038346290588
Global Epoch 2/10
10532/10532 [==============================] - 5s 491us/step - loss: 0.5172 - accuracy: 0.7636
validation accuray: 0.7636083364486694
Global Epoch 3/10
10532/10532 [==============================] - 5s 491us/step - loss: 0.5095 - accuracy: 0.7661
validation accuray: 0.7660504579544067
Global Epoch 4/10
10532/10532 [==============================] - 5s 501us/step - loss: 0.5046 - accuracy: 0.7683
validation accuray: 0.7682996988296509
Global Epoch 5/10
10532/10532 [==============================] - 5s 492us/step - loss: 0.5010 - accuracy: 0.7697
validation accuray: 0.7697388529777527
Global Epoch 6/10
10532/10532 [==============================] - 5s 481us/step - loss: 0.4981 - accuracy: 0.7710
validation accuray: 0.7709643840789795
Global Epoch 7/10
10532/10532 [==============================] - 5s 505us/step - loss: 0

# local and global accuracy, fairness (EOp) of FedAvg

In [6]:
y_val_pred={}
local_EO=[]
def compute_equal_odd(y_true, y_pred, groups):
    white_tpr_1 = np.mean(y_pred[(y_true == 1) & (groups == 0)])
    black_tpr_1 = np.mean(y_pred[(y_true == 1) & (groups == 1)])
    white_tpr_0 = np.mean((1-y_pred)[(y_true == 0) & (groups == 0)])
    black_tpr_0 = np.mean((1-y_pred)[(y_true == 0) & (groups == 1)])
    EO=np.sum([abs(white_tpr_1-black_tpr_1),abs(white_tpr_0-black_tpr_0)])
    if EO<1.01:
        return EO
    else:
        return 'not defined'
    return EO
for state in states:
    y_val_pred[state]=global_model.predict(val_features[state])
    y_val_pred[state]=np.where(y_val_pred[state]<0.5,0,1)
    EO=compute_equal_odd(val_labels[state],y_val_pred[state],vali_groups[state])
    local_EO.append(EO)
all_valid_pred=global_model.predict(all_vaild_features)
all_valid_pred=np.where(all_valid_pred<0.5,0,1)
global_EO=compute_equal_odd(all_vaild_labels,all_valid_pred,all_valid_groups)
print('global EO',global_EO)
print('local EO',local_EO)
local_EO_avg=np.mean(local_EO)
print('local EO average',local_EO_avg)


10532/10532 [==============================] - 5s 435us/step
global EO 0.05726452361870171
local EO [0.14420069059389606, 0.16573523416656966, 0.046650083366677364, 0.08746554128844425, 0.042591685051415806, 0.030765159774143125, 0.09545313921049631, 0.02837433032737635, 0.04717428445779248, 0.06278209380890631, 0.04999283785974418, 0.03619870043678902, 0.06104868528874746, 0.05167364808451047, 0.08131429325887146, 0.06764979225963713, 0.07071157137772344, 0.11942668910237114, 0.0732973414617667, 0.08352189736037768, 0.15580919628262208, 0.12562202815405188, 0.07499373234539036, 0.07524222913410594, 0.08211203311595949, 0.09332149825019859, 0.05744637724590557, 0.04162110282935244, 0.18998946200622271, 0.06323958374649585, 0.06494809506286359, 0.08425054522033854, 0.06327808893535142, 0.24138138148750737, 0.09184068931785527, 0.06089207968988586, 0.08361449582625685, 0.07799185326957975, 0.18798686504446577, 0.08062392909111077, 0.20120721351450466, 0.11993541391672696, 0.0706460927984

In [7]:
acc=global_model.evaluate(all_vaild_features,all_vaild_labels)
print('loss, accuracy:',acc)    

10532/10532 [==============================] - 5s 485us/step - loss: 0.4911 - accuracy: 0.7745
loss, accuracy: [0.4911182224750519, 0.7744896411895752]


# Compute the local statistics for LP

In [8]:
states_for_local_fairness = [
    "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA",
    "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD",
    "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ",
    "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
    "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI"
]

In [9]:
def p_computation(label, sensitive_attribute, y, a):
    label=np.reshape(label,(-1,))
    N=len(label)
    sensitive_attribute=np.reshape(sensitive_attribute,(-1,))# Total number of samples
    # Create a boolean mask for where both conditions are met
    mask = (label == y) & (sensitive_attribute == a)
    # print(mask)
    # print("Number of matching samples:", np.sum(mask))
    
    # Compute the probability
    p_y_ac = np.sum(mask) / N
    return p_y_ac
#community_1
S= np.zeros((50,2,2))
p = np.zeros(50)
l=[]
for c,state in enumerate(states):
    p[c]=len(val_features[state])/len(all_vaild_features)
for c,state in enumerate(states):
    for a in range(2):
        for y in range(2):
            S[c,a,y] =p[c]* p_computation(val_labels[state],vali_groups[state], y, a)




In [10]:
TP= np.zeros((50,2,2))
def compute_TP (y_pred,label, sensitive_attribute, y, a):
    y_pred=np.reshape(y_pred,(-1,))
    label=np.reshape(label,(-1,))
    N=len(y_pred)
    sensitive_attribute=np.reshape(sensitive_attribute,(-1,))# Total number of samples
    # Create a boolean mask for where both conditions are met
    count_1= (y_pred == y) & (label == y) & (sensitive_attribute == a)
    count_2 = (label== y) & (sensitive_attribute == a)
    # print(sum(count_1),sum(count_2))

    
    # Compute the probability
    TP_y_ac = np.sum(count_1) / np.sum(count_2)
    return TP_y_ac

for c, state in enumerate(states):
    y_pred=global_model.predict(val_features[state])
    y_pred=np.where(y_pred<0.5,0,1)
    for a in range(2):
        for y in range(2):
            TP[c,a,y]=compute_TP(y_pred,val_labels[state],vali_groups[state],y,a)
# print(TP)

18/18 [==============================] - 0s 484us/step


In [11]:
def compute_alpha(label, sensitive_attribute, y, a):
    label=np.reshape(label,(-1,))
    N=len(label)
    sensitive_attribute=np.reshape(sensitive_attribute,(-1,))# Total number of samples
    # Create a boolean mask for where both conditions are met
    mask = (label == y) & (sensitive_attribute == a)
    # print(mask)
    # print("Number of matching samples:", np.sum(mask))
    
    # Compute the probability
    alpha_y_ac = np.sum(mask) / N
    return alpha_y_ac
alpha_a_y = np.zeros((2,2))
for a in range(2):
    for y in range(2):
        alpha_a_y[a,y]=compute_alpha(all_vaild_labels,all_valid_groups,y,a)
# print(alpha_a_y)


## Solve the LP

In [21]:
from scipy.optimize import linprog
import numpy as np
def LP_EO(e_0,e_c):
# define the objective function
    C=[]
    for c in range(50):
        for a in range(2):
            for y in range(2):
                C.append(-S[c,a,y])
    #define the global fairness constraints
    N=2
    K=2
    A_1=[]
    A_2=[]
    for c in range(50):
        for a in range(2):
            for y in range(2):
                if  a==0 and y==0:
                    A_1.append(-S[c,a,y]/alpha_a_y[a,y])
                    # print(S[c,a,y],alpha_a_y[a,y])
                elif  a==1 and y==0:
                    A_1.append(S[c,a,y]/alpha_a_y[a,y])
    
                else:
                    A_1.append(0)
    
    for c in range(50):
        for a in range(2):
            for y in range(2):
                if c==c and a==0 and y==1:
                    A_2.append(-S[c,a,y]/alpha_a_y[a,y])
                elif c==c and a==1 and y==1:
                    A_2.append(S[c,a,y]/alpha_a_y[a,y])
                else:
                    A_2.append(0)
    # define local fairness constraints
     # Identity matrix of size 50x50, where each row is a basis vector
    A_1=np.reshape(A_1,(1,200))
    A_2=np.reshape(A_2,(1,200))
    # print(np.shape(A_1),np.shape(A_2))
    # To access e_i, you can use basis_vectors[i], e.g., e_0 is basis_vectors[0]
    
    # Define zero vector in R^50
    zero_vec = np.zeros((2,4))
    I_vector =np.eye(2)
    basis_vectors = np.hstack([I_vector, -I_vector])
    
    # Construct the matrix
    # First row: [e_0, e_1, 0, 0]
    rows = []

# Construct 50 rows dynamically
    for i in range(50):
        # Initialize the blocks for this row
        row_blocks = []
        
        for j in range(50):
            if i == j:  # Diagonal pattern with identity matrices
                row_blocks.append(basis_vectors)
            else:  # All other positions are zero matrices
                row_blocks.append(zero_vec)
        
        # Horizontally stack the blocks for the current row
        row = np.hstack(row_blocks)
        rows.append(row)
    
    # Combine all rows into a single matrix
    A_3 = np.vstack(rows)
    # print(A_3)

    # Combine rows into a single matrix
    # define the property of dervied outcome predictor
    def K_ac_compute(a,c):
        K_ac=np.zeros((3,2))
        l_ac=np.zeros((3,1))
        K_ac[0:]=[-1,-1]
        K_ac[1:]=[(1-TP[c,a,1]), TP[c,a,0]]
        K_ac[2:]=[TP[c,a,1], (1-TP[c,a,0])]
        l_ac[0:]=[-1]
        l_ac[1:]=[TP[c,a,0]]
        l_ac[2:]=[TP[c,a,1]]
        return K_ac,l_ac
    num_clients = 50
    block_rows = 3
    block_cols = 2
    # Define the submatrices k_01, k_11, k_02, k_12 as 3x2 matrices
    M = np.zeros((300,200))
    l = np.zeros((300, 1))
    
    # Construct the matrices for all clients
    for c in range(num_clients):
        for a in range(2):
            # Compute submatrices for client c and attribute a
            K_ac, l_ac = K_ac_compute(a, c)
            
            # Calculate starting positions for placing K_ac in M
            start_row = (2 * c + a) * block_rows
            start_col = (2 * c + a) * block_cols
            
            # Place K_ac and l_ac in the appropriate block of M and l
            M[start_row:start_row + block_rows, start_col:start_col + block_cols] = K_ac
            l[start_row:start_row + block_rows] = l_ac
    
    # Verify the dimensions of the resulting matrix and vector
    # print("Shape of matrix M:", M.shape)
    # print("Shape of vector l:", l.shape)
    # print(np.shape(A_1),np.shape(A_2),np.shape(A_3),np.shape(M))
    # combine the constraints
    print(np.shape(A_1),np.shape(A_2),np.shape(A_3),np.shape(M))
    A=np.vstack((A_1,A_2,A_3, -A_1, -A_2, -A_3,M))
    # print(A)
    b_global=e_0*np.ones((2,1))
    b_local=e_c*np.ones((100,1))
    # print(b_local)
    b=np.vstack((b_global,b_local,b_global,b_local,l))
    # print(b)
    res = linprog(C, A_ub=A, b_ub=b)
    x=res.x
    x=np.reshape(x,(50,2,2))
    # print(x)
    # print(res.fun)
    return x,res.fun
x,fun=LP_EO(0.001,0.001)
                
            

(1, 200) (1, 200) (100, 200) (300, 200)


# Clients solve the LAE and generate fair outcome prediction
 

In [22]:
beta=np.zeros((50,2,3))
for a in range(2):
        for c in range(50):
            A=np.array([[1,1,1],[TP[c,a,0],1,0],[TP[c,a,1],0,1]])
            b=np.array([1,x[c,a,0],x[c,a,1]])
            beta_ac=np.linalg.solve(A,b)
            beta[c,a,:]=beta_ac

def compute_tilde_Y(c,a, Y_hat, y_values):
    """
    Compute \widetilde{Y}_{\boldsymbol{\beta}_{ac}}(x,a,c) based on given probabilities.

    Parameters:
    - beta_ac: Dictionary with keys 'beta_0' and 'beta_y' for probabilities.
    - Y_hat: The predicted value \hat{Y}(x,a,c).
    - y_values: List or array of possible y values in \mathcal{Y}.

    Returns:
    - tilde_Y: The computed value of \widetilde{Y}.
    """
    # Extract probabilities
    beta_0 = beta[c, a, 0]  # Probability for Y_hat
    beta_y = beta[c, a, 1:] 
    # Probabilities for other y in \mathcal{Y}
    
    # Normalize probabilities
    
    # Generate a random number
    rand_val = np.random.rand()
    
    # Determine the output based on random value
    if rand_val < beta_0:
        return Y_hat
    else:
        cumulative_prob = beta_0
        for y in y_values:
            cumulative_prob += beta_y[y]
            if rand_val < cumulative_prob:
                return y
y_tilde_list = [[] for _ in range(50)] 
# Example usage
for c, state in enumerate(states):
    client_features = test_features[state]
    client_sensitive = test_groups[state]
    y_pred=global_model.predict(test_features[state])
    y_pred=np.where(y_pred<0.5,0,1)
    for i in range (len(y_pred)):
        a=int(client_sensitive[i])
        y_hat=int(y_pred[i])
        y_tilde=compute_tilde_Y(c,a,y_hat,[0,1])
        y_tilde=int(y_tilde)
        y_tilde_list[c].append(y_tilde)
# print(y_tilde_list[1])

 1/74 [..............................] - ETA: 0s

/tmp/ipykernel_2278851/2933905974.py:49: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_hat=int(y_pred[i])


12/12 [==============================] - 0s 488us/step


## Fairness and Accuracy after post-processing

In [23]:
local_post_EO=[]
for c,state in enumerate(states):
    y_post=y_tilde_list[c]
    y_post=np.reshape(y_post,(-1,))
    y_true=test_labels[state]
    y_group=test_groups[state]
    dis=compute_equal_odd(y_true,y_post,y_group)
    local_post_EO.append(dis)
print('local post EO',local_post_EO)
local_EO=np.mean(local_post_EO)
print('local EO',local_EO)
y_tilde_list = np.concatenate(y_tilde_list)  

post_EO=compute_equal_odd(all_test_labels,y_tilde_list,all_test_groups)
print('global EO', post_EO)
#compute accuracy
count=np.where(y_tilde_list==all_test_labels,1,0)
accuracy=np.sum(count)/len(count)
print('accuracy',accuracy)

local post EO [0.03475241120389316, 0.0547033755201804, 0.03738355548427974, 0.11791907533687468, 0.008848322132528064, 0.020345774168932185, 0.010696935945617325, 0.07252347680702115, 0.045514527062299415, 0.046641958610895984, 0.12033356117566646, 0.0, 0.048202182654224324, 0.0285040207621865, 0.20663609111374032, 0.0285434543949241, 0.04979720664728432, 0.01542157925832821, 0.014646048292734788, 0.00733496965554381, 0.03335026982304909, 0.016632324595616643, 0.09075465250852283, 0.06184355845991141, 0.06076061065973612, 0.07538827049144259, 0.004750922561407869, 0.08793853986131772, 0.16211377828237775, 0.02868815370990835, 0.027059552559182465, 0.014292969890505935, 0.030861891567245836, 0.0, 0.027132548739157658, 0.014362815026153164, 0.08303074703613395, 0.03025170683448508, 0.1225062392990443, 0.02279535852397252, 0.002673796791443861, 0.08926586672998454, 0.03488968490838551, 0.0, 0.06365145898278429, 0.05169183952667389, 0.05718776448178958, 0.0030963302752293814, 0.0113546179

## Print the results

In [24]:
print('before post processing')
print('local equal opportunity',local_EO_avg)
print('global equal opportunity',global_EO)
print('accuracy',acc)
print('after post processing')
print('local equal opportunity',local_EO)
print('global equal opportunity',post_EO)
print('accuracy',accuracy)

before post processing
local equal opportunity 0.09173882939116375
global equal opportunity 0.05726452361870171
accuracy [0.4911182224750519, 0.7744896411895752]
after post processing
local equal opportunity 0.04554149592661419
global equal opportunity 0.003342479604087567
accuracy 0.7550355582059795
